In [1]:
import os
import json
from typing import List,Union,Optional
import re

markdown_title_types = ["#","##","###","####"]
markdown_title_types = [t +" " for t in markdown_title_types]

def md_split_by_level(texts:List[str],level:Union[int,str]=1)->List[dict]:
    """
    Function：按照level对texts进行拆分，保存level级别的标题以方便补全
    input:
        texts：List[str],逐行读取的md文件
        level：Union[int,str]，1代表一级标题，等价于“#”
    
    output:
        splited_texts：List[dict]
        {
            "title":"level级别对应的title",
            "level":"level",
            "texts":"level级别下对应的文本内容",
        }
    """

    # pre-processing,去掉texts中头尾没有意义的内容
    while texts != [] and re.sub(r"[\n ]*","",texts[0]) =="":
        texts.pop(0)

    while texts != [] and re.sub(r"[\n ]*","",texts[-1]) =="":
        texts.pop()
    

    spliter = markdown_title_types
    level_spliter = spliter[level-1]
    splited_texts = []
    cur_text = []

    index_count = 0

    for line in texts:
        if line.startswith(level_spliter):
            if cur_text!=[]:
                if cur_text[0].startswith(level_spliter):
                    title = cur_text[0]
                    cur_text.pop(0)
                else:
                    title = None
                splited_item = {}
                splited_item["level"] = level
                splited_item["texts"] = cur_text
                if title is not None:
                    title = title.lstrip(level_spliter)
                    title = f"{level_spliter}{index_count+1}.{title}"
                    index_count +=1
                splited_item["title"] = title
                splited_texts.append(splited_item)
                cur_text = []
        cur_text.append(line)
    

    if cur_text !=[]:
        if cur_text[0].startswith(level_spliter):
            title = cur_text[0]
            cur_text.pop(0)
        else:
            title = None
        splited_item = {}
        splited_item["level"] = level
        splited_item["texts"] = cur_text
        # if title is not None:
        #     title = title.lstrip(level_spliter)
        #     title = f"{level_spliter}{index_count+1}.{title}"
            
        splited_item["title"] = title
        splited_texts.append(splited_item)
    return splited_texts

In [3]:
os.getenv("OPENAI_API_KEY") is None

True

In [98]:
def md_split(texts:List[str]):
    """
    function：从一级标题逐级切分文档，并补全对应的标题
    input:
        texts:List[str]，,逐行读取的md文件
    """
    ready_to_split = []
    ready_to_split.append((0,[],texts))
    results = []

    while ready_to_split!=[]:
        item = ready_to_split.pop()
        pre_level = item[0]
        pre_title = item[1]
        ready_texts = item[2]
        if pre_level < 2:
            level_md_split_results = md_split_by_level(ready_texts,pre_level+1)
            for split_item in level_md_split_results:
                ready_to_split.append((pre_level+1,pre_title+[split_item["title"]],split_item["texts"]))
        else:
            titles = [t for t in pre_title if t is not None]
            all_chunk_texts = titles+ready_texts
            all_chunk_texts = [c.strip("\n") for c in all_chunk_texts]
            results.append("\n".join(all_chunk_texts))
    
    return results

In [99]:
test_md_file = []
with open("D:\job\code\RAG\RAG_system\Build-your-own-domain-RAG-base-Milvus\data\HowToCook-master\dishes\meat_dish\回锅肉\回锅肉.md","r",encoding="utf-8") as f:
    for line in f.readlines():
        test_md_file.append(line)

In [100]:
results = md_split(test_md_file)

In [101]:
results

['# 回锅肉的做法\n## 附加内容\n\n- 不喜欢蒜苗可以换成洋葱或者其他蔬菜，但是要注意蔬菜的易熟程度将蔬菜提前炒至，不然会出现蔬菜半生不熟的情况\n- 如果回锅肉比较大块可以切成 5 厘米见方的块，五花肉煮至筷子稍微用力即可插入猪皮即可\n- 回锅肉过冷水晾凉后肉质会更紧致\n- 回锅肉擦干水是为了避免炒至的时候爆油溅伤\n- 回锅肉切记不要切厚了，不然很腻\n- 如果您遵循本指南的制作流程而发现有问题或可以改进的流程，请提出 Issue 或 Pull request 。\n\n如果您遵循本指南的制作流程而发现有问题或可以改进的流程，请提出 Issue 或 Pull request 。',
 '# 回锅肉的做法\n## 3.操作\n\n### 五花肉一段处理\n\n- 锅烧热，用手将五花肉紧紧压在锅上炙皮\n  - 这一步是为了处理猪皮上的汗腺（或者买肉的时候让师傅烧一下皮，喜欢汗腺的可以无视）\n- 用钢丝球把皮洗干净，不洗干净会有苦味\n- 将五花肉放入锅中，放入能淹没五花肉的水，放入生姜片、料酒和小葱（取 2 棵小葱打结）\n- 开大火煮，水开后撇去浮沫，继续煮 15 分钟，煮至瘦肉部分可以用筷子轻松刺穿\n\n### 配菜处理\n\n- 青红椒切圈\n- 蒜苗切段\n- 生姜切小薄片\n- 将 5ml 豆瓣酱和 5ml 生抽提前混合\n\n### 五花肉二段处理\n\n- 将煮熟的五花肉捞出放入冷水晾凉\n- 擦干五花肉的水，切成上肥下瘦的 2mm 的薄片（切厚了口感不好，而且很油）\n\n### 开始炒肉\n\n注意，此步骤**操作要迅速，小心糊锅**\n\n1. 锅烧热，放入一层底油滑锅\n2. 放入五花肉煸炒至肥肉透明，肉片微卷（欲称起灯盏窝），二刀肉效果最佳。\n3. 倒入豆瓣生抽混合物，5g 味精翻炒 15 秒\n4. 放入青红椒圈和小姜片，放入另 5ml 豆瓣酱翻炒 30 秒\n5. 放入蒜苗翻炒 60 秒\n6. 出锅\n\n### 简易版本\n\n- 选用冰冻五花肉常量放置 0.5 小时 或者鲜五花肉放冰箱冷藏 1 个小时，切成 2-5 mm 薄片\n- 开中火，辣椒放过锅中干煸 30-45 秒后取出\n- 锅烧热，放入一层底油滑锅，放入姜片煸炒 15 秒\n- 倒入五花肉，间隔 10 S 翻炒一次，待五花肉出现焦黄色（翻炒时间越久

In [25]:
splited_texts = md_split_by_level(test_md_file,1)

In [26]:
splited_texts

[{'title': '# 回锅肉的做法\n',
  'level': 1,
  'texts': ['\n',
   '预估烹饪难度：★★★★\n',
   '\n',
   '## 必备原料和工具\n',
   '\n',
   '- 五花肉\n',
   '- 小葱\n',
   '- 生姜\n',
   '- 青红椒\n',
   '- 蒜苗\n',
   '- 料酒\n',
   '- 豆瓣酱\n',
   '- 生抽酱油\n',
   '- 味精\n',
   '\n',
   '## 计算\n',
   '\n',
   '- 五花肉的用量为 0.5 斤/男人 0.3 斤/女人 （正宗回锅肉使用二刀肉[俗称：臀尖]制作，肉质坚实，肥瘦合适）\n',
   '- 小葱 2 棵\n',
   '- 生姜 10-40g\n',
   '- 青红椒（根据受辣程度选择, 0-30g）\n',
   '- 蒜苗 1 把\n',
   '- 料酒 5ml\n',
   '- 豆瓣酱 10ml\n',
   '- 味精 5g\n',
   '- 生抽 5ml\n',
   '\n',
   '## 操作\n',
   '\n',
   '### 五花肉一段处理\n',
   '\n',
   '- 锅烧热，用手将五花肉紧紧压在锅上炙皮\n',
   '  - 这一步是为了处理猪皮上的汗腺（或者买肉的时候让师傅烧一下皮，喜欢汗腺的可以无视）\n',
   '- 用钢丝球把皮洗干净，不洗干净会有苦味\n',
   '- 将五花肉放入锅中，放入能淹没五花肉的水，放入生姜片、料酒和小葱（取 2 棵小葱打结）\n',
   '- 开大火煮，水开后撇去浮沫，继续煮 15 分钟，煮至瘦肉部分可以用筷子轻松刺穿\n',
   '\n',
   '### 配菜处理\n',
   '\n',
   '- 青红椒切圈\n',
   '- 蒜苗切段\n',
   '- 生姜切小薄片\n',
   '- 将 5ml 豆瓣酱和 5ml 生抽提前混合\n',
   '\n',
   '### 五花肉二段处理\n',
   '\n',
   '- 将煮熟的五花肉捞出放入冷水晾凉\n',
   '- 擦干五花肉的水，切成上肥下瘦的 2mm 的薄片（切厚了口感不好，而且很油）\n',
   '\n',


In [4]:
md_file_string = "".join(test_md_file)

In [9]:
md_file_string

'# 回锅肉的做法\n\n预估烹饪难度：★★★★\n\n## 必备原料和工具\n\n- 五花肉\n- 小葱\n- 生姜\n- 青红椒\n- 蒜苗\n- 料酒\n- 豆瓣酱\n- 生抽酱油\n- 味精\n\n## 计算\n\n- 五花肉的用量为 0.5 斤/男人 0.3 斤/女人 （正宗回锅肉使用二刀肉[俗称：臀尖]制作，肉质坚实，肥瘦合适）\n- 小葱 2 棵\n- 生姜 10-40g\n- 青红椒（根据受辣程度选择, 0-30g）\n- 蒜苗 1 把\n- 料酒 5ml\n- 豆瓣酱 10ml\n- 味精 5g\n- 生抽 5ml\n\n## 操作\n\n### 五花肉一段处理\n\n- 锅烧热，用手将五花肉紧紧压在锅上炙皮\n  - 这一步是为了处理猪皮上的汗腺（或者买肉的时候让师傅烧一下皮，喜欢汗腺的可以无视）\n- 用钢丝球把皮洗干净，不洗干净会有苦味\n- 将五花肉放入锅中，放入能淹没五花肉的水，放入生姜片、料酒和小葱（取 2 棵小葱打结）\n- 开大火煮，水开后撇去浮沫，继续煮 15 分钟，煮至瘦肉部分可以用筷子轻松刺穿\n\n### 配菜处理\n\n- 青红椒切圈\n- 蒜苗切段\n- 生姜切小薄片\n- 将 5ml 豆瓣酱和 5ml 生抽提前混合\n\n### 五花肉二段处理\n\n- 将煮熟的五花肉捞出放入冷水晾凉\n- 擦干五花肉的水，切成上肥下瘦的 2mm 的薄片（切厚了口感不好，而且很油）\n\n### 开始炒肉\n\n注意，此步骤**操作要迅速，小心糊锅**\n\n1. 锅烧热，放入一层底油滑锅\n2. 放入五花肉煸炒至肥肉透明，肉片微卷（欲称起灯盏窝），二刀肉效果最佳。\n3. 倒入豆瓣生抽混合物，5g 味精翻炒 15 秒\n4. 放入青红椒圈和小姜片，放入另 5ml 豆瓣酱翻炒 30 秒\n5. 放入蒜苗翻炒 60 秒\n6. 出锅\n\n### 简易版本\n\n- 选用冰冻五花肉常量放置 0.5 小时 或者鲜五花肉放冰箱冷藏 1 个小时，切成 2-5 mm 薄片\n- 开中火，辣椒放过锅中干煸 30-45 秒后取出\n- 锅烧热，放入一层底油滑锅，放入姜片煸炒 15 秒\n- 倒入五花肉，间隔 10 S 翻炒一次，待五花肉出现焦黄色（翻炒时间越久五花肉口感越硬）\n- 倒入之前干煸过的辣椒，生抽调味，继续翻炒 60 秒\n- 出锅\n

In [20]:
re.split(r"(\n## [^\n]*)",md_file_string,)

['# 回锅肉的做法\n\n预估烹饪难度：★★★★\n',
 '\n## 必备原料和工具',
 '\n\n- 五花肉\n- 小葱\n- 生姜\n- 青红椒\n- 蒜苗\n- 料酒\n- 豆瓣酱\n- 生抽酱油\n- 味精\n',
 '\n## 计算',
 '\n\n- 五花肉的用量为 0.5 斤/男人 0.3 斤/女人 （正宗回锅肉使用二刀肉[俗称：臀尖]制作，肉质坚实，肥瘦合适）\n- 小葱 2 棵\n- 生姜 10-40g\n- 青红椒（根据受辣程度选择, 0-30g）\n- 蒜苗 1 把\n- 料酒 5ml\n- 豆瓣酱 10ml\n- 味精 5g\n- 生抽 5ml\n',
 '\n## 操作',
 '\n\n### 五花肉一段处理\n\n- 锅烧热，用手将五花肉紧紧压在锅上炙皮\n  - 这一步是为了处理猪皮上的汗腺（或者买肉的时候让师傅烧一下皮，喜欢汗腺的可以无视）\n- 用钢丝球把皮洗干净，不洗干净会有苦味\n- 将五花肉放入锅中，放入能淹没五花肉的水，放入生姜片、料酒和小葱（取 2 棵小葱打结）\n- 开大火煮，水开后撇去浮沫，继续煮 15 分钟，煮至瘦肉部分可以用筷子轻松刺穿\n\n### 配菜处理\n\n- 青红椒切圈\n- 蒜苗切段\n- 生姜切小薄片\n- 将 5ml 豆瓣酱和 5ml 生抽提前混合\n\n### 五花肉二段处理\n\n- 将煮熟的五花肉捞出放入冷水晾凉\n- 擦干五花肉的水，切成上肥下瘦的 2mm 的薄片（切厚了口感不好，而且很油）\n\n### 开始炒肉\n\n注意，此步骤**操作要迅速，小心糊锅**\n\n1. 锅烧热，放入一层底油滑锅\n2. 放入五花肉煸炒至肥肉透明，肉片微卷（欲称起灯盏窝），二刀肉效果最佳。\n3. 倒入豆瓣生抽混合物，5g 味精翻炒 15 秒\n4. 放入青红椒圈和小姜片，放入另 5ml 豆瓣酱翻炒 30 秒\n5. 放入蒜苗翻炒 60 秒\n6. 出锅\n\n### 简易版本\n\n- 选用冰冻五花肉常量放置 0.5 小时 或者鲜五花肉放冰箱冷藏 1 个小时，切成 2-5 mm 薄片\n- 开中火，辣椒放过锅中干煸 30-45 秒后取出\n- 锅烧热，放入一层底油滑锅，放入姜片煸炒 15 秒\n- 倒入五花肉，间隔 10 S 翻炒一次，待五花肉出现焦黄色（翻炒时间越久五花肉口感越硬）\n- 倒入

In [105]:
menue_md_folder_path = "..\..\data\HowToCook-master\dishes"
for root, dirs, files in os.walk(menue_md_folder_path):
    for file in files:
        print(os.path.join(root, file))

..\..\data\HowToCook-master\dishes\aquatic\咖喱炒蟹.md
..\..\data\HowToCook-master\dishes\aquatic\微波葱姜黑鳕鱼.md
..\..\data\HowToCook-master\dishes\aquatic\水煮鱼.md
..\..\data\HowToCook-master\dishes\aquatic\清蒸生蚝.md
..\..\data\HowToCook-master\dishes\aquatic\红烧鱼.md
..\..\data\HowToCook-master\dishes\aquatic\红烧鱼头.md
..\..\data\HowToCook-master\dishes\aquatic\红烧鲤鱼.md
..\..\data\HowToCook-master\dishes\aquatic\芥末黄油罗氏虾.md
..\..\data\HowToCook-master\dishes\aquatic\小龙虾\小龙虾.md
..\..\data\HowToCook-master\dishes\aquatic\小龙虾\成品.jpg
..\..\data\HowToCook-master\dishes\aquatic\干煎阿根廷红虾\干煎阿根廷红虾.jpg
..\..\data\HowToCook-master\dishes\aquatic\干煎阿根廷红虾\干煎阿根廷红虾.md
..\..\data\HowToCook-master\dishes\aquatic\油焖大虾\油焖大虾.jpg
..\..\data\HowToCook-master\dishes\aquatic\油焖大虾\油焖大虾.md
..\..\data\HowToCook-master\dishes\aquatic\混合烤鱼\烤鱼.jpg
..\..\data\HowToCook-master\dishes\aquatic\混合烤鱼\烤鱼.md
..\..\data\HowToCook-master\dishes\aquatic\清蒸鲈鱼\摆盘.jpg
..\..\data\HowToCook-master\dishes\aquatic\清蒸鲈鱼\改刀.jpg
..\..\data\HowToCook-ma

In [106]:
for s,o in zip([1,2],[3,4]):
    print(s,o)

1 3
2 4


In [115]:
s = set([1,2,3,1,4])

In [116]:
list(s)

[1, 2, 3, 4]

In [14]:
import json
with open(r"D:\job\code\RAG\RAG_system\Build-your-own-domain-RAG-base-Milvus\data\documents\retrival_texts_to_documents.json","r",encoding="utf-8") as f:
    data = json.load(f)

In [15]:
len(data)

6863

In [16]:
data[0]

{'retrieval_text': '# 咖喱炒蟹的做法\n## 附加内容\n\n- 做法参考：[十几年澳门厨房佬教学挂汁的咖喱蟹怎么做](https://www.bilibili.com/video/BV1Nq4y1W7K9)',
 'document': '# 咖喱炒蟹的做法\n## 附加内容\n\n- 做法参考：[十几年澳门厨房佬教学挂汁的咖喱蟹怎么做](https://www.bilibili.com/video/BV1Nq4y1W7K9)\n\n如果您遵循本指南的制作流程而发现有问题或可以改进的流程，请提出 Issue 或 Pull request 。'}

In [10]:
s_len = [len(s["retrieval_text"]) for s in data]
s_len.index(2221)

1107

In [9]:
print(data[1107])

# 基础牛奶面包的做法
## 3.操作

### 酵头的制作

酵头是预发酵的产物，通过预发酵，不仅可以提升成功率，而且更长的发酵时间会让面包的风味更好。如果有条件，酵头应该发酵 4 天以达到最佳风味。但对新手来说， 30 分钟的发酵的酵头所呈现的成品就已经很可观了。接下来制作酵头。

![发酵前酵头的理想状态](./4-1酵头1.jpg)

1. 首先，将酵母和 ***30℃ 的温水*** 用刮刀或厨具混合均匀，静置 5 分钟。之后与面粉混合，搅拌均匀。此时的酵头应该是**特别粘稠**的面糊。

![用布盖上酵头](./4-2酵头2.jpg)

2. 用布盖上面糊，将面糊放置到**温暖的地方**进行发酵，时长为 45 ~ 60 分钟。

![酵头的最终形态](./4-3酵头3.jpg)

3. 最终的面糊应该是表面有很多气泡的且体积明显增大。

**发酵失败了？看看这里：**

>1. 确定酵母没有过期
>2. 确定水温合适
>3. 延长发酵时间，可以放在冷藏室内过夜
>4. 可能温度不够，这时可以将烤箱预热至 60℃ ，停止加热后放到烤箱内进行发酵
>5. 可以再加入等量的酵母搅拌均匀再次发酵并延长发酵时间
>6. 可以尝试加入 3g 糖再次发酵

**如何制作“永久的”酵头？**

>有了酵头，随时开烤面包就不是梦！“永久的”酵头可以按下面的方法制作：
>
>1. 按上述方法制作好酵头后，放到阴凉的地方（甚至是冷藏室！）每1-3天在上面撒上薄薄的一层面粉（只要看不见面糊的厚度就可以！）再次搅拌，然后继续盖上布发酵。如果发酵过快体积过大，可以扔掉一部分。
>2. 等到开始制作面包时，只需留下发酵前体积的酵头继续发酵，剩下的用于面包的制作。
>3. 久而久之，酵头就会产生独特的香味，这种香味对于面包来说是最好的“原料”！

### 面团的制作

有了酵头，那就成功一半了！

![此时的面团](./4-4此时的面团.jpg)

1.将剩下的原料全部与酵头用刮刀混合、搅拌。在搅拌时一定要刮壁，使其充分混合。在形成面团且没有干粉时，用干净的干燥的一只手按压面团，另一只手扶住容器，使其形成一个大的面团，然后倒在面板或硅胶垫上操作

>此时地面团应该已经成型并且比较软。如果无法成型且很黏，那么说明面粉较少，可以加入 ¼ cup 面粉再次充分搅拌，直至成型。如果的面团成型了或